In [1]:
import re
import os
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
def fetch(url):
    result = requests.get(url)
    soup = BeautifulSoup(result.content, "html.parser")
    return soup

In [3]:
def get_categories():
    top_category_url = "https://se.trustpilot.com/categories"
    category_soup = fetch(top_category_url)
    tags = category_soup.find("script", {"id":"__NEXT_DATA__"})
    data = json.loads(tags.text)
    tree = data["props"]
    all_categories = []
    cats = tree["pageProps"]["categories"]
    for category in cats:
        all_categories.append(category["categoryId"])
    category_urls = []
    for cat in all_categories:
        category_url = top_category_url + "/" + cat
        category_urls.append(category_url) # All category URLs collected from site
    return category_urls

In [4]:
categories = get_categories()

In [5]:
print(categories)

['https://se.trustpilot.com/categories/animals_pets', 'https://se.trustpilot.com/categories/electronics_technology', 'https://se.trustpilot.com/categories/events_entertainment', 'https://se.trustpilot.com/categories/vehicles_transportation', 'https://se.trustpilot.com/categories/business_services', 'https://se.trustpilot.com/categories/health_medical', 'https://se.trustpilot.com/categories/home_garden', 'https://se.trustpilot.com/categories/hobbies_crafts', 'https://se.trustpilot.com/categories/home_services', 'https://se.trustpilot.com/categories/legal_services_government', 'https://se.trustpilot.com/categories/construction_manufactoring', 'https://se.trustpilot.com/categories/food_beverages_tobacco', 'https://se.trustpilot.com/categories/media_publishing', 'https://se.trustpilot.com/categories/money_insurance', 'https://se.trustpilot.com/categories/travel_vacation', 'https://se.trustpilot.com/categories/restaurants_bars', 'https://se.trustpilot.com/categories/public_local_services', 

In [12]:
data = []

if not os.path.exists("data"):
    os.makedir("data")

for category in categories[0:1]:
    category_dir = category.split("/")
    category_dir = category_dir[len(category_dir)-1]
    if not os.path.exists("data/" + category_dir):
        os.mkdir("data/" + category_dir)
    
    category_specific_companies = []
    category_soup = fetch(category)
    category_tags = category_soup.find("script", {"id":"__NEXT_DATA__"})
    category_data = json.loads(category_tags.text)
    # Category is for example https://se.trustpilot.com/categories/animals_pets
    n_pages = category_data["props"]["pageProps"]["seoData"]["maxPages"]
    for i in range(1, n_pages):
        category_page = category + "?page=" + str(i)
        company_soup = fetch(category_page)
        company_tags = company_soup.find("script", {"id":"__NEXT_DATA__"})
        company_data = json.loads(company_tags.text)
        try: 
            
            #print(company_data)
            company_tree = company_data["props"]["pageProps"]["businessUnits"]
            for k, v in company_tree.items():
                if type(v) == list:
                    for j in v:
                        company_id = j["identifyingName"]
                        if company_id not in category_specific_companies:
                            category_specific_companies.append(company_id)
        except AttributeError:
            print("COMPANY ERROR! There was an error with this company:" + company_id)
        i += 1
    for c in category_specific_companies:
        if not os.path.exists("data/" + category_dir + "/" + c):
            os.mkdir("data/" + category_dir + "/" + c)
        count_soup = fetch("https://se.trustpilot.com/review/" + c)
        count_tags = count_soup.find("script", {"id":"__NEXT_DATA__"})
        count_data = json.loads(count_tags.text)
        count_tree = count_data["props"]
        count_pages = count_tree["pageProps"]["filters"]["pagination"]["totalPages"]
        for page in range(1, count_pages):
            try:
                review_soup = fetch("https://se.trustpilot.com/review/" + c + "?page=" + str(page) +"&stars=3") # Specify rating if important here
                review_tags = review_soup.find("script", {"id":"__NEXT_DATA__"})
                review_data = json.loads(review_tags.text)
                review_tree = review_data["props"]["pageProps"]
                review_tree = review_tree["reviews"]
                review = review_tree[0]["text"]
                rating = review_tree[0]["rating"]
                if rating == 3:
                    data.append((review, rating, category, c))
                    #print((rating, review, c, category)
            except AttributeError:
                print("PAGE ERROR! There was an error on this page: " + "https://se.trustpilot.com/review/" + c + "?page=" + str(page))

PAGE ERROR! There was an error on this page: https://se.trustpilot.com/review/www.propet.se?page=8
PAGE ERROR! There was an error on this page: https://se.trustpilot.com/review/www.propet.se?page=10
PAGE ERROR! There was an error on this page: https://se.trustpilot.com/review/www.propet.se?page=11
PAGE ERROR! There was an error on this page: https://se.trustpilot.com/review/www.propet.se?page=12
PAGE ERROR! There was an error on this page: https://se.trustpilot.com/review/www.propet.se?page=13
PAGE ERROR! There was an error on this page: https://se.trustpilot.com/review/www.propet.se?page=14
PAGE ERROR! There was an error on this page: https://se.trustpilot.com/review/www.propet.se?page=15
PAGE ERROR! There was an error on this page: https://se.trustpilot.com/review/www.propet.se?page=16
PAGE ERROR! There was an error on this page: https://se.trustpilot.com/review/www.propet.se?page=17
PAGE ERROR! There was an error on this page: https://se.trustpilot.com/review/www.propet.se?page=18
P

AttributeError: 'NoneType' object has no attribute 'text'

In [7]:
data

[('Köpte handledskylare för att prova mot vallningar. \nUpplever dock ingen direkt kylande effekt.',
  3,
  'https://se.trustpilot.com/categories/animals_pets',
  'hansesgarden.se'),
 ('Jag blev bemött på ett bra sätt inga konstigheter.',
  3,
  'https://se.trustpilot.com/categories/animals_pets',
  '4dogs.se'),
 ('Alldeles för lång leveranstid!  Varorna låg i Sundsvall i nästan en vecka, från ca kl.06.00 på fredagsmorgonen till nästföljande torsdag. Byt från DHL till Bussgods eller PostNord!',
  3,
  'https://se.trustpilot.com/categories/animals_pets',
  '4dogs.se'),
 ('Snabb leverans och mkt smidigt att beställa.\nBlev lite besviken på de olika kvaliteterna på fällarna. Den ena är otroligt fin och rejäl , medans den andra inte alls håller samma form.\nMan ser inte ens vad mönstret föreställer. 😞',
  3,
  'https://se.trustpilot.com/categories/animals_pets',
  'www.propet.se'),
 ('Bra bra bra bra bra',
  3,
  'https://se.trustpilot.com/categories/animals_pets',
  'www.propet.se'),
 ('S

In [10]:
df = pd.DataFrame(data, columns=["text", "rating", "category", "company"])

In [11]:
df

,text,rating,category,company
0,Köpte handledskylare för att prova mot vallnin...,3,https://se.trustpilot.com/categories/animals_pets,hansesgarden.se
1,Jag blev bemött på ett bra sätt inga konstighe...,3,https://se.trustpilot.com/categories/animals_pets,4dogs.se
2,Alldeles för lång leveranstid! Varorna låg i ...,3,https://se.trustpilot.com/categories/animals_pets,4dogs.se
3,Snabb leverans och mkt smidigt att beställa.\n...,3,https://se.trustpilot.com/categories/animals_pets,www.propet.se
4,Bra bra bra bra bra,3,https://se.trustpilot.com/categories/animals_pets,www.propet.se
5,Snabb leverans som vanligt men jag fick fel pr...,3,https://se.trustpilot.com/categories/animals_pets,svenskadjurapoteket.se
6,"köpte påse med 1000 pälssnoddar, blandade färg...",3,https://se.trustpilot.com/categories/animals_pets,www.haromi.se
